In [84]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [85]:
import re

import geopandas as gpd
import requests,json


from shapely import geos
from shapely.geometry import Point
import fiona
from fiona.crs import from_epsg,from_string

from geopandas import GeoDataFrame
from shapely.geometry import Polygon
from geopandas import GeoSeries


from shapely import geometry
from shapely.geometry import shape,LineString,Point,Polygon

import osmnx as ox
import networkx as nx

import folium

from folium import IFrame

In [86]:
def rest(address):
    try:
        url = 'https://api.tomtom.com/search/2/geocode/{}.json?&key=h2n93GznJsnsDHtv2vPpdT9GmJijp9o9&countrySet=TWN&language=zh-TW&limit=1'.format(address)
        response = requests.get(url)
        data = response.text
        js = json.loads(str(data))
        return js['results'][0]['position']
    except Exception as e:
        print(f'An Error occurred: {e}')
        return {'lat':0,'lon':0}
    
def trans(gdf):
    gdf.crs = {'init' :'epsg:3824'} # 避免資料沒設，這邊再重新給一次
    gdf=gdf.to_crs({'init' :'epsg:4326'})
    return gdf

def utf2asc(s):
    return str(str(s).encode('ascii', 'xmlcharrefreplace'))[2:-1]

heading3 = """<h3>{}</h3>""".format

In [87]:
df_get_back=pd.read_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\get back.xlsx')
print(df_get_back.shape)
df_get_back.head()

df_get_back=df_get_back.loc[:,['id','lon','lat','check']]

(133014, 6)


In [88]:
pop=pd.read_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\台灣202006年村里人口.xlsx')
print(pop.shape)

(7760, 5)


In [89]:
site=['1G30林口中山店']
date=['202007']
start=['190726']

import warnings
warnings.filterwarnings("ignore")

import os
os.chdir('C:/Users/RT009223/TestriteGroup/滲透率專案/01_raw data/TLW社區店/'+site[0]+'/'+date[0])

In [90]:
df01=pd.read_csv(r'ff交易紀錄.csv',header=None,names=['id','site','date','AMT','trans_cnt','tick_cnt','card_type'])
print(df01.shape)
df01.head()

(2170, 7)


,id,site,date,AMT,trans_cnt,tick_cnt,card_type
0,2992019519018,1G30,2020-07-20 00:00:00.000,472.0,1,1,02
1,2992277793014,1G30,2020-07-23 00:00:00.000,380.0,1,1,00
2,2014517310116,1G30,2020-07-23 00:00:00.000,743.0,1,1,00
3,2014919900014,1G30,2020-07-24 00:00:00.000,44.0,1,1,00
4,2616001087018,1G30,2020-07-19 00:00:00.000,380.0,1,1,00


In [91]:
df01['temp']=0
df01.loc[df01.card_type=='05',['temp']]=1

df01_2=df01.groupby(['site','temp'])['AMT'].sum().reset_index(name='AMT')
df01_2['share']=df01_2.AMT/df01_2.AMT.sum()

df_final01=df01_2.copy()
df_final01

,site,temp,AMT,share
0,1G30,0,3102038.06,0.907096
1,1G30,1,317706.76,0.092904


In [92]:
df02=pd.read_csv(r'一般卡會員活躍程度.csv',header=None,names=['id','type'])
print(df02.shape)
df02.head()

(1648, 2)


,id,type
0,19960129000234,2
1,19960129000912,2
2,19960201000012,2
3,19960201000259,2
4,19960201000469,2


In [93]:
df02.type=df02.type.apply(lambda x:str(x))
df02.type=df02.type.replace('1','type1:new')
df02.type=df02.type.replace('2','type2:active')
df02.type=df02.type.replace('3','type3:1yr~3yr')
df02.type=df02.type.replace('4','type4:3yr~')

df02_2=df02.groupby(['type'])['id'].count().reset_index(name='n')
df02_2['share']=df02_2.n/df02_2.n.sum()

df_final02=df02_2.copy()
df_final02

,type,n,share
0,type1:new,56,0.033981
1,type2:active,1193,0.723908
2,type3:1yr~3yr,126,0.076456
3,type4:3yr~,273,0.165655


In [94]:
df03=pd.read_csv(r'一般卡會員會員樣貌.csv',header=None,names=['id','name','gender','birth','status','age','age_type'])
print(df03.shape)
df03.head()

(1648, 7)


,id,name,gender,birth,status,age,age_type
0,19961020000057,邱培宏,M,1968-07-07 00:00:00.000,2,52,09:50-54
1,19961025000015,鄭滿欽,M,1957-02-26 00:00:00.000,1,63,11:60-64
2,19961025000062,鍾興文,M,1900-01-01 00:00:00.000,2,120,99:其他
3,20110627000390,李志清,M,1968-07-01 00:00:00.000,1,52,09:50-54
4,20110710000332,徐鳳珠,F,1963-09-01 00:00:00.000,1,56,10:55-59


In [95]:
df03=pd.merge(df03,df02,on='id')

df03_2=df03.groupby(['age_type'])['id'].count().reset_index(name='n_all')
df03_2['share_all']=df03_2.n_all/df03_2.n_all.sum()
df03_3=df03[df03.type=='type1:new'].groupby(['age_type'])['id'].count().reset_index(name='n_new')
df03_3['share_new']=df03_3.n_new/df03_3.n_new.sum()

df03_4=pd.merge(df03_2,df03_3,on='age_type',how='outer')
df03_4=df03_4.fillna(0)
print(df03_4.share_all.sum())
print(df03_4.share_new.sum())

df_final03=df03_4.copy()
df_final03

1.0
1.0


,age_type,n_all,share_all,n_new,share_new
0,02:15-19,2,0.001214,0.0,0.000000
1,03:20-24,22,0.013350,12.0,0.214286
2,04:25-29,65,0.039442,5.0,0.089286
3,05:30-34,93,0.056432,4.0,0.071429
4,06:35-39,195,0.118325,6.0,0.107143
5,07:40-44,266,0.161408,3.0,0.053571
6,08:45-49,227,0.137743,4.0,0.071429
7,09:50-54,228,0.138350,4.0,0.071429
8,10:55-59,191,0.115898,6.0,0.107143
9,11:60-64,127,0.077063,3.0,0.053571


In [96]:
df03_5=df03.groupby(['gender'])['id'].count().reset_index(name='n_all')
df03_5['share_all']=df03_5.n_all/df03_5.n_all.sum()
df03_6=df03[df03.type=='type1:new'].groupby(['gender'])['id'].count().reset_index(name='n_new')
df03_6['share_new']=df03_6.n_new/df03_6.n_new.sum()

df03_7=pd.merge(df03_5,df03_6,on='gender',how='outer')
df03_7=df03_7.fillna(0)
print(df03_7.share_all.sum())
print(df03_7.share_new.sum())

df_final04=df03_7.copy()
df_final04

1.0
1.0


,gender,n_all,share_all,n_new,share_new
0,F,635,0.385316,24.0,0.428571
1,M,1012,0.614078,32.0,0.571429
2,Z,1,0.000607,0.0,0.000000


In [97]:
df04=pd.read_csv(r'sales一般卡交易紀錄.csv',header=None,names=['id','card','site','date','sku','class1','class1_name','AMT'])
print(df04.shape)
df04.head()

(6098, 8)


,id,card,site,date,sku,class1,class1_name,AMT
0,19960206000347,2010379230142,1G30,2020-07-12 00:00:00.000,139366,18,家用品,228.0
1,19960303000391,2010600400184,1G30,2020-07-02 00:00:00.000,9134264,18,家用品,199.0
2,19960706000017,2011051640150,1G30,2020-07-29 00:00:00.000,16103139,21,電器/電料,33.0
3,19961116000055,2011451160142,1G30,2020-07-30 00:00:00.000,195786,15,門窗,486.0
4,19970624000009,2011985110118,1G30,2020-07-29 00:00:00.000,206154,20,層架/收納,26.0


In [98]:
df04_2=pd.merge(df04,df03,on='id',how='left')
print(df04_2.shape)
df04_2.head()

(6098, 15)


,id,card,site,date,sku,class1,class1_name,AMT,name,gender,birth,status,age,age_type,type
0,19960206000347,2010379230142,1G30,2020-07-12 00:00:00.000,139366,18,家用品,228.0,楊宗豫,M,1960-02-25 00:00:00.000,1,60,11:60-64,type2:active
1,19960303000391,2010600400184,1G30,2020-07-02 00:00:00.000,9134264,18,家用品,199.0,喬治,M,1967-12-05 00:00:00.000,1,52,09:50-54,type2:active
2,19960706000017,2011051640150,1G30,2020-07-29 00:00:00.000,16103139,21,電器/電料,33.0,江杰,M,1960-01-01 00:00:00.000,1,60,11:60-64,type2:active
3,19961116000055,2011451160142,1G30,2020-07-30 00:00:00.000,195786,15,門窗,486.0,卜翠娟,F,1900-01-01 00:00:00.000,1,120,99:其他,type2:active
4,19970624000009,2011985110118,1G30,2020-07-29 00:00:00.000,206154,20,層架/收納,26.0,林賜平,M,1900-01-01 00:00:00.000,1,120,99:其他,type3:1yr~3yr


In [99]:
df04_3=df04_2.groupby(['class1_name'])['id'].nunique().reset_index(name='id_all')
df04_3['id_all_rate']=df04_3.id_all/df03.shape[0]
print(df04_3.shape)

df04_4=df04_2[df04_2.type=='type1:new'].groupby(['class1_name'])['id'].nunique().reset_index(name='id_new')
df04_4['id_new_rate']=df04_4.id_new/df03[df03.type=='type1:new'].shape[0]
print(df04_4.shape)
df04_4.head()

(25, 3)
(18, 3)


,class1_name,id_new,id_new_rate
0,五金,4,0.071429
1,休閒樂活&園藝,5,0.089286
2,傢俱,2,0.035714
3,地材,3,0.053571
4,家用品,16,0.285714


In [100]:
dept_name=pd.read_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\01_raw data\TLW社區店\大分類中文名稱.xlsx')
df04_5=pd.merge(df04_3,df04_4,on='class1_name',how='outer')

df_final05=pd.merge(dept_name,df04_5,how='left')
df_final05=df_final05.fillna(0)
print(df_final05.shape)
df_final05.head()

(92, 5)


,class1_name,id_all,id_all_rate,id_new,id_new_rate
0,油漆/防水,170.0,0.103155,6.0,0.107143
1,膠類,128.0,0.077670,1.0,0.017857
2,窗飾/壁飾,41.0,0.024879,0.0,0.000000
3,地材,55.0,0.033374,3.0,0.053571
4,燈源/燈具,208.0,0.126214,11.0,0.196429


In [101]:
df_01=pd.read_csv('一般卡會員(有完整地址)_'+start[0]+'.csv',error_bad_lines=False,header=None)
df_01.columns=['id','addr','lon','lat']
df_01.shape

(12749, 4)

In [102]:
pattern = re.compile(r'(..[市,縣])(.{1,2}[市,區,鄉,鎮])(.{2,3}[里,村])')
location=df_01.addr.apply(lambda x:pattern.findall(x))

addr01=[]
addr02=[]
addr03=[]
for i in location:
    if len(i)==1:
        addr01.append(i[0][0])
        addr02.append(i[0][1])
        addr03.append(i[0][2])        
    else:
        addr01.append(np.nan)
        addr02.append(np.nan)
        addr03.append(np.nan)  

In [103]:
df_01['縣市']=addr01
df_01['行政區']=addr02
df_01['村里']=addr03

In [104]:
df_02=df_01.dropna()
df_02.shape

(12747, 7)

In [105]:
gdf=gpd.read_file(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\geo data\VILLAGE_MOI_1090423.shp',encoding='utf-8')
gdf=gdf.loc[:,['COUNTYNAME','TOWNNAME','VILLNAME','geometry']]
gdf=trans(gdf)
gdf=gdf.dropna().reset_index(drop=True)
print(gdf.shape)

gdf.columns=['縣市','行政區','村里','geometry']
gdf['縣市']=list(gdf['縣市'].apply(lambda x:x.replace('臺','台')))
gdf['行政區']=list(gdf['行政區'].apply(lambda x:x.replace('臺','台')))
gdf['村里']=list(gdf['村里'].apply(lambda x:x.replace('臺','台')))

df_02['縣市']=list(df_02['縣市'].apply(lambda x:x.replace('臺','台')))
df_02['行政區']=list(df_02['行政區'].apply(lambda x:x.replace('臺','台')))
df_02['村里']=list(df_02['村里'].apply(lambda x:x.replace('臺','台')))

gdf['村里']=gdf['村里'].apply(lambda x:x.replace('舘','館'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('舘','館'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('双','雙'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('双','雙'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))

gdf['村里']=gdf['村里'].apply(lambda x:x.replace('欍','瓊'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('溝垻里','溝埧里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('崙峰里','崙峯里'))
gdf.head()

(7760, 4)


,縣市,行政區,村里,geometry
0,高雄市,林園區,中門里,"POLYGON ((120.36772 22.49564, 120.36427 22.498..."
1,高雄市,林園區,港埔里,"POLYGON ((120.37325 22.49123, 120.37147 22.492..."
2,高雄市,旗津區,上竹里,"POLYGON ((120.28976 22.57317, 120.28577 22.577..."
3,高雄市,旗津區,中興里,"POLYGON ((120.30320 22.55568, 120.30284 22.557..."
4,連江縣,南竿鄉,津沙村,"POLYGON ((119.91684 26.14109, 119.91683 26.141..."


In [106]:
site1=df_02[-(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].groupby(['村里'])['行政區'].nunique().reset_index(name='n').sort_values(by='n',ascending=False).reset_index(drop=True)
site1.head(5)

,村里,n
0,?寮里,1
1,?廷里,1
2,?林里,1
3,?洲里,1
4,?福村,1


In [107]:
for i in site1[site1.n>0]['村里']:
    print(i, df_02[df_02['村里']==i]['行政區'].unique())

?寮里 ['樹林區']
?廷里 ['土城區']
?林里 ['大園區']
?洲里 ['永康區']
?福村 ['民雄鄉']
?行里 ['永康區']
上?里 ['竹東鎮']
下?里 ['彰化市']
五?里 ['新店區']
公?里 ['板橋區']
公西里 ['龜山區' '林口區']
南?里 ['彰化市']
新?里 ['永和區']
永?里 ['三峽區']
瓦?里 ['中和區']
糖?里 ['萬華區']


In [108]:
site2=site1[site1.n>0]['村里'][0]
site3=df_02[df_02['村里']==site2]['行政區'].unique()

for i in site3:
    print(i,list(gdf[gdf['行政區']==i]['村里'][gdf[gdf['行政區']==i]['村里'].str.contains('里')]))

樹林區 ['南園里', '柑園里', '西園里', '東園里', '北園里', '彭厝里', '太順里', '彭興里', '山佳里', '中華里', '彭福里', '大同里', '樂山里', '東陽里', '東昇里', '和平里', '中山里', '樹南里', '育英里', '樹人里', '樹西里', '樹東里', '樹北里', '樹興里', '樹福里', '樹德里', '東山里', '保安里', '坡內里', '文林里', '潭底里', '圳福里', '金寮里', '光興里', '圳安里', '獇寮里', '圳民里', '三多里', '三興里', '三福里', '圳生里', '三龍里']


In [109]:
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('糖部里','糖廍里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('糖?里','糖廍里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('磚?里','磚磘里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('磚瑤里','磚磘里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('下?里','下廍里'))

df_02.loc[df_02['行政區']=='彰化市',['村里']]=df_02.loc[df_02['行政區']=='彰化市',['村里']].apply(lambda x:x.replace('南?里','南瑶里'))
df_02.loc[df_02['行政區']=='官田區',['村里']]=df_02.loc[df_02['行政區']=='官田區',['村里']].apply(lambda x:x.replace('南?里','南廍里'))
#df_02['村里']=df_02['村里'].apply(lambda x:x.replace('南?里','南瑶里'))


df_02['村里']=df_02['村里'].apply(lambda x:x.replace('崙?里','崙峯里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?子里','廍子里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('石?村','石硦村'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('上?里','上館里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('永?里','永館里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('瓦?村','瓦磘村'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('瓦?里','瓦磘里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('五?里','五峯里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?行里','塩行里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?廷里','峯廷里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?興里','塩興里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?廷里','峯廷里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('公?里','公館里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?洲里','塩洲里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?龍里','雙龍里'))

#df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?林里','雞林里'))
df_02.loc[df_02['行政區']=='竹東鎮',['村里']]=df_02.loc[df_02['行政區']=='竹東鎮',['村里']].apply(lambda x:x.replace('?林里','雞林里'))

df_02.loc[df_02['行政區']=='大園區',['村里']]=df_02.loc[df_02['行政區']=='大園區',['村里']].apply(lambda x:x.replace('?林里','菓林里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?寮里','獇寮里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('新?里','新廍里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('蔗?里','蔗廍里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?埔里','坔埔里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?榔里','槺榔里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?江里','晋江里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?福村','雙福村'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('爪?里','爪峯里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('舊?村','舊館村'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('南?村','南館村'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('灰?里','灰磘里'))
df_02['村里']=df_02['村里'].apply(lambda x:x.replace('三?村','三廍村'))


df_02.loc[df_02['行政區']=='將軍區',['村里']]=df_02.loc[df_02['行政區']=='將軍區',['村里']].apply(lambda x:x.replace('忠興里','忠嘉里'))

df_02['村里']=df_02['村里'].apply(lambda x:x.replace('下部里','下廍里'))
df_02.loc[df_02['行政區']=='埔鹽鄉',['村里']]=df_02.loc[df_02['行政區']=='埔鹽鄉',['村里']].apply(lambda x:x.replace('?子村','廍子村'))
df_02.loc[df_02['行政區']=='四湖鄉',['村里']]=df_02.loc[df_02['行政區']=='四湖鄉',['村里']].apply(lambda x:x.replace('?子村','萡子村'))

In [110]:
#合
df_02_1=df_02[(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)

#不合
df_02_2=df_02[-(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
print(df_02_1.shape)
print(df_02_2.shape)

(12746, 7)
(1, 7)


In [111]:
df_02_2.lon=df_02_2.lon.apply(lambda x:float(x))
df_02_2.lat=df_02_2.lat.apply(lambda x:float(x))

In [112]:
if df_02_2.shape[0]>0:
    geom=[Point(data.lon,data.lat) for idx, data in df_02_2.iterrows()]
    crs = {'init': 'epsg:4326'}
    df_02_2 = gpd.GeoDataFrame(df_02_2, crs=crs, geometry=geom)
    print(df_02_2.shape)
    df_02_3 = gpd.tools.sjoin(df_02_2,gdf, how="inner", op='intersects')
    print(df_02_3.shape)
    df_02_3=df_02_3.iloc[:,[0,1,2,3,9,10,11]].reset_index(drop=True)
    df_02_3.columns=['id','addr','lon','lat','縣市','行政區','村里']
    df_02_3.head()
else:
    df_02_3=df_02_2.copy()

(1, 8)
(1, 12)


In [113]:
df_02=pd.concat([df_02_1,df_02_3]).reset_index(drop=True)
df_02.shape

(12747, 7)

In [114]:
df_03=df_01[-df_01.id.isin(df_02.id)].dropna(subset=['lon'])
df_03.shape

(2, 7)

In [115]:
pattern = re.compile(r'(..[市,縣])(.{2,3}[市,區,鄉,鎮])(.{1,2}[里,村])')
location=df_03.addr.apply(lambda x:pattern.findall(x))

addr01=[]
addr02=[]
addr03=[]
for i in location:
    if len(i)==1:
        addr01.append(i[0][0])
        addr02.append(i[0][1])
        addr03.append(i[0][2])        
    else:
        addr01.append(np.nan)
        addr02.append(np.nan)
        addr03.append(np.nan)  

In [116]:
df_03['縣市']=addr01
df_03['行政區']=addr02
df_03['村里']=addr03

In [117]:
#合
df_03_1=df_03[(df_03['縣市']+df_03['行政區']+df_03['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
print(df_03_1.shape)

#不合
df_03_2=df_03[-(df_03['縣市']+df_03['行政區']+df_03['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
print(df_03_2.shape)


(1, 7)
(1, 7)


In [118]:
def to_float(x):
    try:
        return float(x)
    except:
        return 0

In [119]:
df_03_2.lon=df_03_2.lon.apply(to_float)
df_03_2.lat=df_03_2.lat.apply(to_float)

if df_03_2.shape[0]>0:
    geom=[Point(data.lon,data.lat) for idx, data in df_03_2.iterrows()]
    crs = {'init': 'epsg:4326'}
    df_03_2 = gpd.GeoDataFrame(df_03_2, crs=crs, geometry=geom)
    print(df_03_2.shape)
    df_03_3 = gpd.tools.sjoin(df_03_2,gdf, how="inner", op='intersects')
    print(df_03_3.shape)
    df_03_3=df_03_3.iloc[:,[0,1,2,3,9,10,11]].reset_index(drop=True)
    df_03_3.columns=['id','addr','lon','lat','縣市','行政區','村里']
    print(df_03_3.shape)
else:
    df_03_3=df_03_2.copy()

(1, 8)
(1, 12)
(1, 7)


In [120]:
df_03=pd.concat([df_03_1,df_03_3]).reset_index(drop=True)

In [121]:
df_01=pd.concat([df_02,df_03]).reset_index(drop=True)
df_01.shape

(12749, 7)

In [122]:
df_04=pd.read_excel(r'一般卡會員(無完整地址)_'+start[0]+'.xlsx')
df_04.shape

(2461, 3)

In [123]:
df_04['addr']=df_04.addr.apply(lambda x:str(x))

In [124]:
df_04_1=df_04[df_04.addr.apply(lambda x:len(x)>9)].reset_index(drop=True)
df_04_1.shape

(1310, 3)

In [125]:
pattern = re.compile(r'..[市,縣].+[路,街].+')
location=df_04_1.addr.apply(lambda x:pattern.findall((x)))

addr01=[]
for i in location:
    if len(i)==1:
        addr01.append(i[0])      
    else:
        addr01.append(np.nan)

In [126]:
df_04_1.addr=addr01
df_04_1=df_04_1.dropna(subset=['addr']).reset_index(drop=True)
df_04_1.shape

(1085, 3)

In [127]:
df_04_1['lon']=float(0)
df_04_1['lat']=float(0)
df_04_1['check']=0
df_04_1.head()

,id,addr,zone,lon,lat,check
0,20010625000563,桃園市大園區2鄰?林路9之47號,337.0,0.0,0.0,0
1,20030318032544,台北市北投區志遠一路2段112巷8號3樓,112.0,0.0,0.0,0
2,19991127002048,台南市歸仁區民權路52,711.0,0.0,0.0,0
3,20180721000698,台北市松山區松山路675之11,105.0,0.0,0.0,0
4,20041017000355,新北市八里區訓塘路６６號,249.0,0.0,0.0,0


In [128]:
df_04_1=pd.merge(df_04_1.loc[:,['id','addr','zone']],df_get_back,on='id',how='left')

In [129]:
df_04_1.lon[df_04_1.lon.isna()]=0
df_04_1.lat[df_04_1.lat.isna()]=0
df_04_1.check[df_04_1.check.isna()]=0

In [130]:
df_04_1.head()

,id,addr,zone,lon,lat,check
0,20010625000563,桃園市大園區2鄰?林路9之47號,337.0,0.00000,0.00000,0.0
1,20030318032544,台北市北投區志遠一路2段112巷8號3樓,112.0,121.51356,25.11144,1.0
2,19991127002048,台南市歸仁區民權路52,711.0,0.00000,0.00000,0.0
3,20180721000698,台北市松山區松山路675之11,105.0,0.00000,0.00000,0.0
4,20041017000355,新北市八里區訓塘路６６號,249.0,0.00000,0.00000,0.0


In [131]:
df_zone=pd.read_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\郵遞區代碼.xlsx')

In [132]:
df_04_1=pd.merge(df_04_1,df_zone,on='zone',how='left')

In [133]:
geom=[Point(data.lon,data.lat) for idx, data in df_04_1.iterrows()]
crs = {'init': 'epsg:4326'}
df_04_1 = gpd.GeoDataFrame(df_04_1, crs=crs, geometry=geom)
df_04_1.shape

(1085, 9)

In [134]:
df_04_1= gpd.tools.sjoin(df_04_1,gdf, how="left", op='intersects')
df_04_1.shape

(1085, 13)

In [135]:
df_04_1.head()

,id,addr,zone,lon,lat,check,縣市_left,行政區_left,geometry,index_right,縣市_right,行政區_right,村里
0,20010625000563,桃園市大園區2鄰?林路9之47號,337.0,0.00000,0.00000,0.0,桃園市,大園區,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN
1,20030318032544,台北市北投區志遠一路2段112巷8號3樓,112.0,121.51356,25.11144,1.0,台北市,北投區,POINT (121.51356 25.11144),7632.0,台北市,北投區,福興里
2,19991127002048,台南市歸仁區民權路52,711.0,0.00000,0.00000,0.0,台南市,歸仁區,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN
3,20180721000698,台北市松山區松山路675之11,105.0,0.00000,0.00000,0.0,台北市,松山區,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN
4,20041017000355,新北市八里區訓塘路６６號,249.0,0.00000,0.00000,0.0,新北市,八里區,POINT (0.00000 0.00000),NaN,NaN,NaN,NaN


In [136]:
# 無 map 無 zone
df_04_1_1=df_04_1[(df_04_1['行政區_left'].isna()&df_04_1['村里'].isna())]
print(df_04_1_1.shape)

# 無 map 有 zone
df_04_1_2=df_04_1[(-(df_04_1['行政區_left'].isna())&(df_04_1['村里'].isna()))]
print(df_04_1_2.shape)

# 有 map 無有 zone
df_04_1_3=df_04_1[(df_04_1['行政區_left'].isna()&(-df_04_1['村里'].isna()))]
print(df_04_1_3.shape)

# 有 map 有 zone
df_04_1_4=df_04_1[(-(df_04_1['行政區_left'].isna())&(-df_04_1['村里'].isna()))]
print(df_04_1_4.shape)

(12, 13)
(833, 13)
(10, 13)
(230, 13)


In [137]:
df_04_1_4_1=df_04_1_4[(df_04_1_4['行政區_left']==df_04_1_4['行政區_right'])]
print(df_04_1_4_1.shape)

df_04_1_4_2=df_04_1_4[-(df_04_1_4['行政區_left']==df_04_1_4['行政區_right'])]
print(df_04_1_4_2.shape)

(196, 13)
(34, 13)


In [138]:
df_04_1_1=df_04_1_1.loc[:,['id']]

df_04_1_2=df_04_1_2.loc[:,['id','縣市_left','行政區_left']]
df_04_1_2.columns=['id','縣市','行政區']

df_04_1_3=df_04_1_3.loc[:,['id','縣市_right','行政區_right','村里']]
df_04_1_3.columns=['id','縣市','行政區','村里']

df_04_1_4_1=df_04_1_4_1.loc[:,['id','縣市_right','行政區_right','村里']]
df_04_1_4_1.columns=['id','縣市','行政區','村里']

df_04_1_4_2=df_04_1_4_2.loc[:,['id','縣市_left','行政區_left']]
df_04_1_4_2.columns=['id','縣市','行政區']

In [139]:
df_04_2=pd.concat([df_04_1_4_1,df_04_1_3,df_04_1_4_2,df_04_1_2,df_04_1_1])
df_04_2.shape

(1085, 4)

In [140]:
df_04_2.drop_duplicates(subset=['id']).shape

(1085, 4)

In [141]:
df_04_3=df_04[-df_04.id.isin(df_04_2.id)]
df_04_3.shape

(1376, 3)

In [142]:
df_04_3=pd.merge(df_04_3,df_zone,on='zone',how='left')

In [143]:
df_04_3_1=df_04_3[-df_04_3['行政區'].isna()]
print(df_04_3_1.shape)

df_04_3_2=df_04_3[df_04_3['行政區'].isna()]
print(df_04_3_2.shape)

(416, 5)
(960, 5)


In [144]:
df_04_3_1=df_04_3_1.loc[:,['id','縣市','行政區']]
df_04_3_2=df_04_3_2.loc[:,['id']]

In [145]:
df_04=pd.concat([df_04_2,df_04_3_1,df_04_3_2])
print(df_04.shape)
print(df_04.drop_duplicates(subset=['id']).shape)

(2461, 4)
(2461, 4)


In [146]:
result=pd.concat([df_01,df_04]).reset_index(drop=True)
result.loc[result['縣市'].isna(),['縣市']]='NA'
result.loc[result['行政區'].isna(),['行政區']]='NA'
result.loc[result['村里'].isna(),['村里']]='NA'
print(result.shape)
result.head()

(15210, 7)


,id,addr,lon,lat,縣市,行政區,村里
0,20000516024253,新北市林口區林口里2鄰仁愛路一段223號 ...,121.397865,25.074855,新北市,林口區,林口里
1,19971227000953,新北市林口區南勢里31鄰民有街68號 ...,121.372886,25.077085,新北市,林口區,南勢里
2,20010227015450,新北市新店區中正里3鄰中正路117巷6號 ...,121.541713,24.970789,新北市,新店區,中正里
3,19970827000013,桃園市龜山區文化里29鄰文興路128號 ...,121.365343,25.056846,桃園市,龜山區,文化里
4,20051216000225,新北市林口區湖北里11鄰後湖19號之3 ...,121.387347,25.098217,新北市,林口區,湖北里


In [147]:
pop['縣市']=pop['縣市'].str.replace('\xa0','')
pop['行政區']=pop['行政區'].str.replace('\xa0','')
pop['村里']=pop['村里'].str.replace('\xa0','')

In [148]:
gdf['縣市']=list(gdf['縣市'].apply(lambda x:x.replace('臺','台')))
gdf['行政區']=list(gdf['行政區'].apply(lambda x:x.replace('臺','台')))
gdf['村里']=list(gdf['村里'].apply(lambda x:x.replace('臺','台')))
pop['縣市']=list(pop['縣市'].apply(lambda x:x.replace('臺','台')))
pop['行政區']=list(pop['行政區'].apply(lambda x:x.replace('臺','台')))
pop['村里']=list(pop['村里'].apply(lambda x:x.replace('臺','台')))

gdf['村里']=gdf['村里'].apply(lambda x:x.replace('舘','館'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('舘','館'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('双','雙'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('双','雙'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))

gdf['村里']=gdf['村里'].apply(lambda x:x.replace('欍','瓊'))

pop['村里']=pop['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))

pop['村里']=pop['村里'].apply(lambda x:x.replace('欍','瓊'))


pop['村里']=pop['村里'].apply(lambda x:x.replace('溝垻里','溝埧里'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('崙峰里','崙峯里'))

In [149]:
result[-(result['行政區']+result['村里']).isin(gdf['行政區']+gdf['村里'])]['村里'].unique()

array(['NA'], dtype=object)

In [150]:
result['村里']=result['村里'].apply(lambda x:x.replace('舘','館'))
result['村里']=result['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
result['村里']=result['村里'].apply(lambda x:x.replace('双','雙'))
result['村里']=result['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
result['村里']=result['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
result['村里']=result['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))
result['村里']=result['村里'].apply(lambda x:x.replace('欍','瓊'))

In [151]:
result2=result.groupby(['縣市','行政區','村里'])['id'].count().reset_index()
result2.columns=['縣市','行政區','村里','會員數']
result2.shape

(2478, 4)

In [152]:
result.shape[0]

15210

In [153]:
result2=pd.merge(pop,result2,how='outer')

In [154]:
result2=pd.merge(pop,result2,how='outer')
result2=result2[-(-result2['村里'].isin(pop['村里'])&-(result2['村里']=='NA'))].reset_index(drop=True)
result2.shape

(7881, 6)

In [155]:
result2[-(result2['行政區']+result2['村里']).isin(gdf['行政區']+gdf['村里'])]['村里'].unique()

array(['NA'], dtype=object)

In [156]:
result2['會員數'].sum()

15210.0

In [157]:
gdf_final=pd.merge(gdf,result2,how='right')

In [158]:
gdf_final['會員數'][gdf_final['會員數'].isna()]=0

In [159]:
gdf_final['site']=gdf_final['行政區']+'\n'+gdf_final['村里']
gdf_final['ratio']=gdf_final['會員數']/gdf_final['戶口']
print(gdf_final.shape)
gdf_final.head()

(7881, 9)


,縣市,行政區,村里,geometry,戶口,人口,會員數,site,ratio
0,高雄市,林園區,中門里,"POLYGON ((120.36772 22.49564, 120.36427 22.498...",1124.0,2884.0,0.0,林園區\n中門里,0.000000
1,高雄市,林園區,港埔里,"POLYGON ((120.37325 22.49123, 120.37147 22.492...",822.0,2054.0,0.0,林園區\n港埔里,0.000000
2,高雄市,旗津區,上竹里,"POLYGON ((120.28976 22.57317, 120.28577 22.577...",1386.0,3228.0,1.0,旗津區\n上竹里,0.000722
3,高雄市,旗津區,中興里,"POLYGON ((120.30320 22.55568, 120.30284 22.557...",1069.0,2886.0,0.0,旗津區\n中興里,0.000000
4,連江縣,南竿鄉,津沙村,"POLYGON ((119.91684 26.14109, 119.91683 26.141...",111.0,441.0,0.0,南竿鄉\n津沙村,0.000000


In [160]:
north1=['新北市','台北市','桃園市']

In [161]:
area=[north1]

In [162]:
location=area[0]
gdf_final_sub=gdf_final.loc[gdf_final['縣市'].isin(location),['縣市','行政區','村里','會員數','戶口','ratio']]
gdf_final_sub.shape

(2041, 6)

In [163]:
df_final06=gdf_final_sub.groupby(['縣市','行政區'])['會員數','戶口'].sum().reset_index()
df_final06['ratio']=df_final06['會員數']/df_final06['戶口']
df_final06=pd.concat([df_final06,pd.DataFrame({'縣市':['其他地區'],'會員數':[result.shape[0]-df_final06['會員數'].sum()]})])
print(df_final06.shape)
df_final06.head()

(55, 5)


,縣市,行政區,會員數,戶口,ratio
0,台北市,中山區,283.0,100714.0,0.002810
1,台北市,中正區,127.0,65731.0,0.001932
2,台北市,信義區,143.0,89130.0,0.001604
3,台北市,內湖區,279.0,109606.0,0.002545
4,台北市,北投區,196.0,98022.0,0.002000


In [164]:
df_final07=gdf_final_sub.copy()
df_final07=pd.concat([df_final07,pd.DataFrame({'縣市':['其他地區'],'會員數':[result.shape[0]-df_final07['會員數'].sum()]})])
print(df_final07.shape)
df_final07.head()

(2042, 6)


,縣市,行政區,村里,會員數,戶口,ratio
42,桃園市,大溪區,仁義里,0.0,1372.0,0.000000
43,桃園市,八德區,瑞德里,0.0,1579.0,0.000000
872,桃園市,龜山區,楓樹里,12.0,1649.0,0.007277
873,桃園市,桃園區,大興里,4.0,2606.0,0.001535
874,桃園市,桃園區,忠義里,6.0,1664.0,0.003606


In [165]:
df_final01['店點']=site[0]
df_final01=df_final01.set_index('店點').reset_index()

df_final02['店點']=site[0]
df_final02=df_final02.set_index('店點').reset_index()

df_final03['店點']=site[0]
df_final03=df_final03.set_index('店點').reset_index()

df_final04['店點']=site[0]
df_final04=df_final04.set_index('店點').reset_index()

df_final05['店點']=site[0]
df_final05=df_final05.set_index('店點').reset_index()

df_final06['店點']=site[0]
df_final06=df_final06.set_index('店點').reset_index()

df_final07['店點']=site[0]
df_final07=df_final07.set_index('店點').reset_index()

In [166]:
df_final05_2=df_final05.copy()
df_final05_2['']=''
df_final05_2=df_final05_2.set_index('').reset_index()


df_final06_2=df_final06.copy()
df_final06_2['']=''
df_final06_2=df_final06_2.set_index('').reset_index()

df_final07_2=df_final07.copy()
df_final07_2['']=''
df_final07_2=df_final07_2.set_index('').reset_index()

In [167]:
df_final05a=df_final05.copy()
df_final06a=df_final06.copy()
df_final07a=df_final07.copy()
df_final05b=df_final05_2.copy()
df_final06b=df_final06_2.copy()
df_final07b=df_final07_2.copy()

In [168]:
df_final06

,店點,縣市,行政區,會員數,戶口,ratio
0,1G30林口中山店,台北市,中山區,283.0,100714.0,0.002810
1,1G30林口中山店,台北市,中正區,127.0,65731.0,0.001932
2,1G30林口中山店,台北市,信義區,143.0,89130.0,0.001604
3,1G30林口中山店,台北市,內湖區,279.0,109606.0,0.002545
4,1G30林口中山店,台北市,北投區,196.0,98022.0,0.002000
5,1G30林口中山店,台北市,南港區,81.0,47975.0,0.001688
6,1G30林口中山店,台北市,士林區,276.0,107984.0,0.002556
7,1G30林口中山店,台北市,大同區,93.0,52049.0,0.001787
8,1G30林口中山店,台北市,大安區,214.0,121457.0,0.001762
9,1G30林口中山店,台北市,文山區,142.0,107424.0,0.001322


In [169]:
site=['1G30林口中山店','1G32桃園大業店','1G33竹北文興店','1G34蘆洲集賢店','1G35板橋新埔店','1G36新營金華店','1G37永康復國店','1G38大安安和店','1G39頭份中央店']
start=['190726','190903','191123','191204','200328','200321','200411','200530','200711']

north1=['新北市','台北市','桃園市']
north2=['新竹縣','新竹市','桃園市','新北市']
north3=['新竹縣','新竹市','桃園市']
north4=['新竹縣','新竹市','苗栗縣']
west=['南投縣','彰化縣','台中市']
south1=['台南市','嘉義縣','嘉義市']
east=[]
area=[north1,north2,north3,north1,north1,south1,south1,north1,north4]

In [170]:
print(int(date[0][2:])*100+40)
end_site=(np.array(list(map(int,start)))<int(date[0][2:])*100+40).sum()
end_site

200740


9

In [171]:
for i in range(1,end_site):
    print(site[i])

1G32桃園大業店
1G33竹北文興店
1G34蘆洲集賢店
1G35板橋新埔店
1G36新營金華店
1G37永康復國店
1G38大安安和店
1G39頭份中央店


# 看不懂的迴圈起始點

In [172]:
for a in range(1,end_site):
    os.chdir('C:/Users/RT009223/TestriteGroup/滲透率專案/01_raw data/TLW社區店/'+site[a]+'/'+date[0])
    print(site[a],date[0])
    df01=pd.read_csv(r'ff交易紀錄.csv',header=None,names=['id','site','date','AMT','trans_cnt','tick_cnt','card_type'])
    print(df01.shape)
    df01.head()
    df01['temp']=0
    df01.loc[df01.card_type=='05',['temp']]=1
    df01_2=df01.groupby(['site','temp'])['AMT'].sum().reset_index(name='AMT')
    df01_2['share']=df01_2.AMT/df01_2.AMT.sum()
    df_final01_another=df01_2.copy()


    df02=pd.read_csv(r'一般卡會員活躍程度.csv',header=None,names=['id','type'])
    print('當月一般卡會員',df02.shape)
    df02.head()
    df02.type=df02.type.apply(lambda x:str(x))
    df02.type=df02.type.replace('1','type1:new')
    df02.type=df02.type.replace('2','type2:active')
    df02.type=df02.type.replace('3','type3:1yr~3yr')
    df02.type=df02.type.replace('4','type4:3yr~')
    df02_2=df02.groupby(['type'])['id'].count().reset_index(name='n')
    df02_2['share']=df02_2.n/df02_2.n.sum()
    df_final02_another=df02_2.copy()

    df03=pd.read_csv(r'一般卡會員會員樣貌.csv',header=None,names=['id','name','gender','birth','status','age','age_type'])
    print(df03.shape)
    df03.head()
    df03=pd.merge(df03,df02,on='id')
    df03_2=df03.groupby(['age_type'])['id'].count().reset_index(name='n_all')
    df03_2['share_all']=df03_2.n_all/df03_2.n_all.sum()
    df03_3=df03[df03.type=='type1:new'].groupby(['age_type'])['id'].count().reset_index(name='n_new')
    df03_3['share_new']=df03_3.n_new/df03_3.n_new.sum()
    df03_4=pd.merge(df03_2,df03_3,on='age_type',how='outer')
    df03_4=df03_4.fillna(0)
    print(df03_4.share_all.sum())
    print(df03_4.share_new.sum())
    df_final03_another=df03_4.copy()

    df03_5=df03.groupby(['gender'])['id'].count().reset_index(name='n_all')
    df03_5['share_all']=df03_5.n_all/df03_5.n_all.sum()
    df03_6=df03[df03.type=='type1:new'].groupby(['gender'])['id'].count().reset_index(name='n_new')
    df03_6['share_new']=df03_6.n_new/df03_6.n_new.sum()
    df03_7=pd.merge(df03_5,df03_6,on='gender',how='outer')
    df03_7=df03_7.fillna(0)
    print(df03_7.share_all.sum())
    print(df03_7.share_new.sum())
    df_final04_another=df03_7.copy()

    df04=pd.read_csv(r'sales一般卡交易紀錄.csv',header=None,names=['id','card','site','date','sku','class1','class1_name','AMT'])
    print(df04.shape)
    df04.head()
    df04_2=pd.merge(df04,df03,on='id',how='left')
    print(df04_2.shape)
    df04_2.head()
    df04_3=df04_2.groupby(['class1_name'])['id'].nunique().reset_index(name='id_all')
    df04_3['id_all_rate']=df04_3.id_all/df03.shape[0]
    print(df04_3.shape)
    df04_3.head()
    df04_4=df04_2[df04_2.type=='type1:new'].groupby(['class1_name'])['id'].nunique().reset_index(name='id_new')
    df04_4['id_new_rate']=df04_4.id_new/df03[df03.type=='type1:new'].shape[0]
    print(df04_4.shape)
    df04_4.head()
    dept_name=pd.read_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\01_raw data\TLW社區店\大分類中文名稱.xlsx')
    df04_5=pd.merge(df04_3,df04_4,on='class1_name',how='outer')
    df04_6=pd.merge(dept_name,df04_5,how='left')
    df04_6=df04_6.fillna(0)
    df_final05_another=df04_6.copy()



    df_01=pd.read_csv('一般卡會員(有完整地址)_'+start[a]+'.csv',error_bad_lines=False,header=None)
    df_01.columns=['id','addr','lon','lat']
    print('一般卡會員(有完整地址)',df_01.shape)
    pattern = re.compile(r'(..[市,縣])(.{1,2}[市,區,鄉,鎮])(.{2,3}[里,村])')
    location=df_01.addr.apply(lambda x:pattern.findall(x))

    addr01=[]
    addr02=[]
    addr03=[]
    for i in location:
        if len(i)==1:
            addr01.append(i[0][0])
            addr02.append(i[0][1])
            addr03.append(i[0][2])        
        else:
            addr01.append(np.nan)
            addr02.append(np.nan)
            addr03.append(np.nan)  
    df_01['縣市']=addr01
    df_01['行政區']=addr02
    df_01['村里']=addr03
    df_02=df_01.dropna()
    print(df_02.shape)

    print(gdf.shape)

    gdf.columns=['縣市','行政區','村里','geometry']
    gdf['縣市']=list(gdf['縣市'].apply(lambda x:x.replace('臺','台')))
    gdf['行政區']=list(gdf['行政區'].apply(lambda x:x.replace('臺','台')))
    gdf['村里']=list(gdf['村里'].apply(lambda x:x.replace('臺','台')))

    df_02['縣市']=list(df_02['縣市'].apply(lambda x:x.replace('臺','台')))
    df_02['行政區']=list(df_02['行政區'].apply(lambda x:x.replace('臺','台')))
    df_02['村里']=list(df_02['村里'].apply(lambda x:x.replace('臺','台')))

    gdf['村里']=gdf['村里'].apply(lambda x:x.replace('舘','館'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('舘','館'))
    gdf['村里']=gdf['村里'].apply(lambda x:x.replace('双','雙'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('双','雙'))
    gdf['村里']=gdf['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
    gdf['村里']=gdf['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
    gdf['村里']=gdf['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
    gdf['村里']=gdf['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))

    gdf['村里']=gdf['村里'].apply(lambda x:x.replace('欍','瓊'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('溝垻里','溝埧里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('崙峰里','崙峯里'))
    gdf.head()
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('糖部里','糖廍里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('糖?里','糖廍里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('磚?里','磚磘里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('磚瑤里','磚磘里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('下?里','下廍里'))

    df_02.loc[df_02['行政區']=='彰化市',['村里']]=df_02.loc[df_02['行政區']=='彰化市',['村里']].apply(lambda x:x.replace('南?里','南瑶里'))
    df_02.loc[df_02['行政區']=='官田區',['村里']]=df_02.loc[df_02['行政區']=='官田區',['村里']].apply(lambda x:x.replace('南?里','南廍里'))



    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('崙?里','崙峯里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?子里','廍子里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('石?村','石硦村'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('上?里','上館里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('永?里','永館里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('瓦?村','瓦磘村'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('瓦?里','瓦磘里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('五?里','五峯里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?行里','塩行里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?廷里','峯廷里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?興里','塩興里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?廷里','峯廷里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('公?里','公館里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?洲里','塩洲里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?龍里','雙龍里'))

    #df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?林里','雞林里'))
    df_02.loc[df_02['行政區']=='竹東鎮',['村里']]=df_02.loc[df_02['行政區']=='竹東鎮',['村里']].apply(lambda x:x.replace('?林里','雞林里'))

    df_02.loc[df_02['行政區']=='大園區',['村里']]=df_02.loc[df_02['行政區']=='大園區',['村里']].apply(lambda x:x.replace('?林里','菓林里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?寮里','獇寮里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('新?里','新廍里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('蔗?里','蔗廍里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?埔里','坔埔里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?榔里','槺榔里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?江里','晋江里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('?福村','雙福村'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('爪?里','爪峯里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('舊?村','舊館村'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('南?村','南館村'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('灰?里','灰磘里'))
    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('三?村','三廍村'))


    df_02.loc[df_02['行政區']=='將軍區',['村里']]=df_02.loc[df_02['行政區']=='將軍區',['村里']].apply(lambda x:x.replace('忠興里','忠嘉里'))

    df_02['村里']=df_02['村里'].apply(lambda x:x.replace('下部里','下廍里'))
    df_02.loc[df_02['行政區']=='埔鹽鄉',['村里']]=df_02.loc[df_02['行政區']=='埔鹽鄉',['村里']].apply(lambda x:x.replace('?子村','廍子村'))
    df_02.loc[df_02['行政區']=='四湖鄉',['村里']]=df_02.loc[df_02['行政區']=='四湖鄉',['村里']].apply(lambda x:x.replace('?子村','萡子村'))

    #合
    df_02_1=df_02[(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)

    #不合
    df_02_2=df_02[-(df_02['縣市']+df_02['行政區']+df_02['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
    print(df_02_1.shape)
    print(df_02_2.shape)
    df_02_2.lon=df_02_2.lon.apply(lambda x:float(x))
    df_02_2.lat=df_02_2.lat.apply(lambda x:float(x))
    if df_02_2.shape[0]>0:        
        geom=[Point(data.lon,data.lat) for idx, data in df_02_2.iterrows()]
        crs = {'init': 'epsg:4326'}
        df_02_2 = gpd.GeoDataFrame(df_02_2, crs=crs, geometry=geom)
        print(df_02_2.shape)
        df_02_3 = gpd.tools.sjoin(df_02_2,gdf, how="inner", op='intersects')
        print(df_02_3.shape)
        df_02_3=df_02_3.iloc[:,[0,1,2,3,9,10,11]].reset_index(drop=True)
        df_02_3.columns=['id','addr','lon','lat','縣市','行政區','村里']
        df_02_3.head()
    else:
        df_02_3=df_02_2.copy()
    df_02=pd.concat([df_02_1,df_02_3]).reset_index(drop=True)
    print(df_02.shape)
    df_03=df_01[-df_01.id.isin(df_02.id)].dropna(subset=['lon'])
    print(df_03.shape)
    pattern = re.compile(r'(..[市,縣])(.{2,3}[市,區,鄉,鎮])(.{1,2}[里,村])')
    location=df_03.addr.apply(lambda x:pattern.findall(x))

    addr01=[]
    addr02=[]
    addr03=[]
    for i in location:
        if len(i)==1:
            addr01.append(i[0][0])
            addr02.append(i[0][1])
            addr03.append(i[0][2])        
        else:
            addr01.append(np.nan)
            addr02.append(np.nan)
            addr03.append(np.nan)  
    df_03['縣市']=addr01
    df_03['行政區']=addr02
    df_03['村里']=addr03
    #合
    df_03_1=df_03[(df_03['縣市']+df_03['行政區']+df_03['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
    print(df_03_1.shape)

    #不合
    df_03_2=df_03[-(df_03['縣市']+df_03['行政區']+df_03['村里']).isin(gdf['縣市']+gdf['行政區']+gdf['村里'])].reset_index(drop=True)
    print(df_03_2.shape)
    def to_float(x):
        try:
            return float(x)
        except:
            return 0
    df_03_2.lon=df_03_2.lon.apply(to_float)
    df_03_2.lat=df_03_2.lat.apply(to_float)
    if df_03_2.shape[0]>0:
        geom=[Point(data.lon,data.lat) for idx, data in df_03_2.iterrows()]
        crs = {'init': 'epsg:4326'}
        df_03_2 = gpd.GeoDataFrame(df_03_2, crs=crs, geometry=geom)
        print(df_03_2.shape)
        df_03_3 = gpd.tools.sjoin(df_03_2,gdf, how="inner", op='intersects')
        print(df_03_3.shape)
        df_03_3=df_03_3.iloc[:,[0,1,2,3,9,10,11]].reset_index(drop=True)
        df_03_3.columns=['id','addr','lon','lat','縣市','行政區','村里']
        print(df_03_3.shape)
    else:
        df_03_3=df_03_2.copy()
    df_03=pd.concat([df_03_1,df_03_3]).reset_index(drop=True)
    df_01=pd.concat([df_02,df_03]).reset_index(drop=True)
    print('一般卡會員(有完整地址)_new',df_01.shape)
    
    
    df_04=pd.read_excel('一般卡會員(無完整地址)_'+start[a]+'.xlsx')
    print('一般卡會員(無完整地址)',df_04.shape)
    df_04['addr']=df_04.addr.apply(lambda x:str(x))
    df_04_1=df_04[df_04.addr.apply(lambda x:len(x)>9)].reset_index(drop=True)
    print(df_04_1.shape)
    pattern = re.compile(r'..[市,縣].+[路,街].+')
    location=df_04_1.addr.apply(lambda x:pattern.findall((x)))

    addr01=[]
    for i in location:
        if len(i)==1:
            addr01.append(i[0])      
        else:
            addr01.append(np.nan)
    df_04_1.addr=addr01
    df_04_1=df_04_1.dropna(subset=['addr']).reset_index(drop=True)
    print(df_04_1.shape)
    df_04_1['lon']=float(0)
    df_04_1['lat']=float(0)
    df_04_1['check']=0
    df_04_1.head()


    print(df_get_back.shape)
    df_get_back.head()
    df_04_1=pd.merge(df_04_1.loc[:,['id','addr','zone']],df_get_back,on='id',how='left')
    df_04_1.lon[df_04_1.lon.isna()]=0
    df_04_1.lat[df_04_1.lat.isna()]=0
    df_04_1.check[df_04_1.check.isna()]=0
    df_04_1=pd.merge(df_04_1,df_zone,on='zone',how='left')
    geom=[Point(data.lon,data.lat) for idx, data in df_04_1.iterrows()]
    crs = {'init': 'epsg:4326'}
    df_04_1 = gpd.GeoDataFrame(df_04_1, crs=crs, geometry=geom)
    df_04_1.shape
    df_04_1= gpd.tools.sjoin(df_04_1,gdf, how="left", op='intersects')
    print(df_04_1.shape)

    # 無 map 無 zone
    df_04_1_1=df_04_1[(df_04_1['行政區_left'].isna()&df_04_1['村里'].isna())]
    print(df_04_1_1.shape)

    # 無 map 有 zone
    df_04_1_2=df_04_1[(-(df_04_1['行政區_left'].isna())&(df_04_1['村里'].isna()))]
    print(df_04_1_2.shape)

    # 有 map 無有 zone
    df_04_1_3=df_04_1[(df_04_1['行政區_left'].isna()&(-df_04_1['村里'].isna()))]
    print(df_04_1_3.shape)

    # 有 map 有 zone
    df_04_1_4=df_04_1[(-(df_04_1['行政區_left'].isna())&(-df_04_1['村里'].isna()))]
    print(df_04_1_4.shape)

    df_04_1_4_1=df_04_1_4[(df_04_1_4['行政區_left']==df_04_1_4['行政區_right'])]
    print(df_04_1_4_1.shape)

    df_04_1_4_2=df_04_1_4[-(df_04_1_4['行政區_left']==df_04_1_4['行政區_right'])]
    print(df_04_1_4_2.shape)

    df_04_1_1=df_04_1_1.loc[:,['id']]

    df_04_1_2=df_04_1_2.loc[:,['id','縣市_left','行政區_left']]
    df_04_1_2.columns=['id','縣市','行政區']

    df_04_1_3=df_04_1_3.loc[:,['id','縣市_right','行政區_right','村里']]
    df_04_1_3.columns=['id','縣市','行政區','村里']

    df_04_1_4_1=df_04_1_4_1.loc[:,['id','縣市_right','行政區_right','村里']]
    df_04_1_4_1.columns=['id','縣市','行政區','村里']

    df_04_1_4_2=df_04_1_4_2.loc[:,['id','縣市_left','行政區_left']]
    df_04_1_4_2.columns=['id','縣市','行政區']

    df_04_2=pd.concat([df_04_1_4_1,df_04_1_3,df_04_1_4_2,df_04_1_2,df_04_1_1])
    print(df_04_2.shape)
    print(df_04_2.drop_duplicates(subset=['id']).shape)
    df_04_3=df_04[-df_04.id.isin(df_04_2.id)]
    print(df_04_3.shape)

    df_04_3=pd.merge(df_04_3,df_zone,on='zone',how='left')
    df_04_3_1=df_04_3[-df_04_3['行政區'].isna()]
    print(df_04_3_1.shape)

    df_04_3_2=df_04_3[df_04_3['行政區'].isna()]
    print(df_04_3_2.shape)
    df_04_3_1=df_04_3_1.loc[:,['id','縣市','行政區']]
    df_04_3_2=df_04_3_2.loc[:,['id']]
    df_04=pd.concat([df_04_2,df_04_3_1,df_04_3_2])
    print('一般卡會員(無完整地址)_new',df_04.shape)
    print(df_04.drop_duplicates(subset=['id']).shape)
    result=pd.concat([df_01,df_04]).reset_index(drop=True)
    result.loc[result['縣市'].isna(),['縣市']]='NA'
    result.loc[result['行政區'].isna(),['行政區']]='NA'
    result.loc[result['村里'].isna(),['村里']]='NA'
    print('全部一般卡會員_new',result.shape)
    result.head()

    result['村里']=result['村里'].apply(lambda x:x.replace('舘','館'))
    result['村里']=result['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
    result['村里']=result['村里'].apply(lambda x:x.replace('双','雙'))
    result['村里']=result['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
    result['村里']=result['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
    result['村里']=result['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))
    result['村里']=result['村里'].apply(lambda x:x.replace('欍','瓊'))

    result2=result.groupby(['縣市','行政區','村里'])['id'].count().reset_index()
    result2.columns=['縣市','行政區','村里','會員數']
    print(result2.shape)
    result2=pd.merge(pop,result2,how='outer')
    result2=result2[-(-result2['村里'].isin(pop['村里'])&-(result2['村里']=='NA'))].reset_index(drop=True)
    print(result2.shape)

    gdf_final=pd.merge(gdf,result2,how='right')
    gdf_final['會員數'][gdf_final['會員數'].isna()]=0
    gdf_final['site']=gdf_final['行政區']+'\n'+gdf_final['村里']
    gdf_final['ratio']=gdf_final['會員數']/gdf_final['戶口']
    print(gdf_final.shape)
    gdf_final.head()
    location=area[a]
    gdf_final_sub=gdf_final.loc[gdf_final['縣市'].isin(location),['縣市','行政區','村里','會員數','戶口','ratio']]
    print(gdf_final_sub.shape)

    df_final06_another=gdf_final_sub.groupby(['縣市','行政區'])['會員數','戶口'].sum().reset_index()
    df_final06_another['ratio']=df_final06_another['會員數']/df_final06_another['戶口']
    df_final06_another=pd.concat([df_final06_another,pd.DataFrame({'縣市':['其他地區'],'會員數':[result.shape[0]-df_final06_another['會員數'].sum()]})])
    print(df_final06_another.shape)

    df_final07_another=gdf_final_sub.copy()
    df_final07_another=pd.concat([df_final07_another,pd.DataFrame({'縣市':['其他地區'],'會員數':[result.shape[0]-df_final07_another['會員數'].sum()]})])
    print(df_final07_another.shape)

    df_final01_another['店點']=site[a]
    df_final01_another=df_final01_another.set_index('店點').reset_index()

    df_final02_another['店點']=site[a]
    df_final02_another=df_final02_another.set_index('店點').reset_index()

    df_final03_another['店點']=site[a]
    df_final03_another=df_final03_another.set_index('店點').reset_index()

    df_final04_another['店點']=site[a]
    df_final04_another=df_final04_another.set_index('店點').reset_index()

    df_final05_another['店點']=site[a]
    df_final05_another=df_final05_another.set_index('店點').reset_index()

    df_final06_another['店點']=site[a]
    df_final06_another=df_final06_another.set_index('店點').reset_index()

    df_final07_another['店點']=site[a]
    df_final07_another=df_final07_another.set_index('店點').reset_index()
    
    df_final05_2=df_final05_another.copy()
    df_final05_2['']=''
    df_final05_2=df_final05_2.set_index('').reset_index()
    
    df_final06_2=df_final06_another.copy()
    df_final06_2['']=''
    df_final06_2=df_final06_2.set_index('').reset_index()

    df_final07_2=df_final07_another.copy()
    df_final07_2['']=''
    df_final07_2=df_final07_2.set_index('').reset_index()

    df_final01=pd.concat([df_final01,df_final01_another])
    df_final02=pd.concat([df_final02,df_final02_another])
    df_final03=pd.concat([df_final03,df_final03_another])
    df_final04=pd.concat([df_final04,df_final04_another])
    
    df_final05a=pd.concat([df_final05a,df_final05_another])
    df_final06a=pd.concat([df_final06a,df_final06_another])
    df_final07a=pd.concat([df_final07a,df_final07_another])

    df_final05b=pd.concat([df_final05b, df_final05_2], axis=1)
    df_final06b=pd.concat([df_final06b, df_final06_2], axis=1)
    df_final07b=pd.concat([df_final07b, df_final07_2], axis=1)
    print('------------------------------------------------------')
    print('------------------------------------------------------')
    print('------------------------------------------------------')

1G32桃園大業店 202007
(2118, 7)
當月一般卡會員 (1594, 2)
(1594, 7)
1.0
1.0000000000000002
0.9999999999999999
1.0
(4642, 8)
(4642, 15)
(24, 3)
(21, 3)
一般卡會員(有完整地址) (8148, 4)
(8147, 7)
(7760, 4)
(8147, 7)
(0, 7)
(8147, 7)
(1, 7)
(1, 7)
(0, 7)
一般卡會員(有完整地址)_new (8148, 7)
一般卡會員(無完整地址) (2508, 3)
(1108, 3)
(978, 3)
(133014, 4)
(978, 13)
(3, 13)
(852, 13)
(6, 13)
(117, 13)
(93, 13)
(24, 13)
(978, 4)
(978, 4)
(1530, 3)
(233, 5)
(1297, 5)
一般卡會員(無完整地址)_new (2508, 4)
(2508, 4)
全部一般卡會員_new (10656, 7)
(1708, 4)
(7853, 6)
(7853, 9)
(1887, 6)
(59, 5)
(1888, 6)
------------------------------------------------------
------------------------------------------------------
------------------------------------------------------
1G33竹北文興店 202007
(2487, 7)
當月一般卡會員 (1960, 2)
(1960, 7)
0.9999999999999998
0.9999999999999998
0.9999999999999999
1.0
(7359, 8)
(7359, 15)
(28, 3)
(22, 3)
一般卡會員(有完整地址) (10713, 4)
(10711, 7)
(7760, 4)
(10706, 7)
(5, 7)
(5, 8)
(5, 12)
(10711, 7)
(2, 7)
(1, 7)
(1, 7)
(1, 8)
(1, 12)
(1, 7)
一般卡會員(有完整地址

In [173]:
date[0]

'202007'

In [174]:
date[0][2:]

'2007'

In [175]:
writer = pd.ExcelWriter('C:/Users/RT009223/TestriteGroup/滲透率專案/06_TLW社區店/month/'+date[0][2:]+'/社區店分析_分析用.xlsx', engine='xlsxwriter')

df_final01.to_excel(writer, sheet_name='臨時卡業績占比',index=0)
df_final02.to_excel(writer, sheet_name='會員組成',index=0)
df_final03.to_excel(writer, sheet_name='年齡',index=0)
df_final04.to_excel(writer, sheet_name='性別',index=0)
df_final05a.to_excel(writer, sheet_name='品類',index=0)
df_final06a.to_excel(writer, sheet_name='區滲透',index=0)
df_final07a.to_excel(writer, sheet_name='里滲透',index=0)

writer.save()

In [176]:
writer = pd.ExcelWriter('C:/Users/RT009223/TestriteGroup/滲透率專案/06_TLW社區店/month/'+date[0][2:]+'/社區店分析('+date[0]+')_Iris用.xlsx', engine='xlsxwriter')

df_final01.to_excel(writer, sheet_name='臨時卡業績占比',index=0)
df_final02.to_excel(writer, sheet_name='會員組成',index=0)
df_final03.to_excel(writer, sheet_name='年齡',index=0)
df_final04.to_excel(writer, sheet_name='性別',index=0)
df_final05b.to_excel(writer, sheet_name='品類',index=0)
df_final06b.to_excel(writer, sheet_name='區滲透',index=0)
df_final07b.to_excel(writer, sheet_name='里滲透',index=0)

writer.save()

In [93]:
location=north1
gdf_final_sub=gdf_final.loc[gdf_final['縣市'].isin(location),['縣市','行政區','村里','會員數','戶口','ratio']]
gdf_final_sub.shape

(2007, 6)